In [ ]:
# 导入所需的库
import torch
import torch.nn as nn
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 设置随机种子，保证结果的可重复性
SEED_NUM = 1037
random.seed(SEED_NUM)
np.random.seed(SEED_NUM)
torch.manual_seed(SEED_NUM)

In [ ]:
# 配置信息
dtype = torch.float32
learning_rate = 0.1
print_every_epoch = 100
SHOW_PICTURE = False
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

在 [−10, 10] × [−10, 10] 的 2D 平面内，以均匀分布随机生成 5000 个数据点(x, y)

In [ ]:
# 生成数据
num_points = 5000  # 数据点个数
x = np.random.uniform(low=-10, high=10, size=num_points)
y = np.random.uniform(low=-10, high=10, size=num_points)
z = x ** 2 + x * y + y ** 2

In [ ]:
# 作图
if SHOW_PICTURE:
    fig = plt.figure()
    ax3 = plt.axes(projection='3d')
    X, Y = np.meshgrid(x, y)
    Z = X ** 2 + X * Y + Y ** 2
    ax3.plot_surface(X, Y, Z, cmap='rainbow')
    plt.show()

In [ ]:
# 将数据转换为张量
x = torch.tensor(x, dtype=dtype).unsqueeze(1)
y = torch.tensor(y, dtype=dtype).unsqueeze(1)
z = torch.tensor(z, dtype=dtype).unsqueeze(1)

In [ ]:
# 划分训练集和数据集
num_train = int(num_points * 0.9)
num_test = num_points - num_train
indices = np.random.permutation(num_points)
train_indices, test_indices = indices[:num_train], indices[num_train:]
x_train, y_train, z_train = x[train_indices], y[train_indices], z[train_indices]
x_test, y_test, z_test = x[test_indices], y[test_indices], z[test_indices]

定义一个至少包含一个隐藏层的前馈神经网络

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Linear(2, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, x_in, y_in):
        t = torch.cat((x_in, y_in), dim=1)
        out = self.conv(t)
        return out


In [ ]:
# 创建模型实例和优化器
model = MyNet()
# model = model.cuda()

In [ ]:
# * 损失函数设置
criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()

In [ ]:
# * 优化器设置
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 训练网络
num_epoches = 1000


def train_net(epoches=num_epoches):
    for epoch in range(num_epoches):
        x_train.to(device=device, dtype=dtype)
        y_train.to(device=device, dtype=dtype)
        outputs = model(x_train, y_train)
        loss = criterion(outputs, z_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % print_every_epoch == 0:
            print(f"Epoch: {epoch}\tloss: {loss}")


# 测试网络
def check_accuracy():
    with torch.no_grad():
        pred = model(x_test, y_test)
        test_loss = criterion(pred, z_test)
    print(f"Test loss: {test_loss.item():.4f}")
    return pred

In [ ]:
train_net()
z_pred = check_accuracy()

In [ ]:
# 绘制预测结果与真实结果之间的散点图
fig, ax = plt.subplots()
ax.scatter(z_test.numpy(), z_pred.numpy())
ax.plot([-50, 300], [-50, 300], "r--")
ax.set_xlabel("True values")
ax.set_ylabel("Predictions")
plt.show()